# Práctica 5: Automatic Differentation (AD) in Python

Automatic Differentation (AD) está implementada en varias librerias de Python: por ejemplo, en TensorFlow aparece en el módulo [tf.autodiff](https://www.tensorflow.org/api_docs/python/tf/autodiff) y en Pytorch en el módulo [TORCH.AUTOGRAD](https://pytorch.org/docs/stable/autograd.html). Por supuesto, AD también está presente (aunque de manera subyacente) en los modelos del módulo [scikit-learn](https://scikit-learn.org/stable/) dedicado a Machine Learning.


[Autograd](https://autograd.readthedocs.io/en/latest/usage.html) es otra pequeña librería, basada en Numpy, que resulta muy conveniente para entender las bases de AD.

En esta práctica nos centraremos en el estudio de los métodos implementados en TensorFlow y Pytorch (que actualmente son las librerías de Machine Learning más utilizadas) para calcular gradientes via AD. Veremos una primera introducción a backward AD en TensorFlow y dejaremos para el ejercicio de esta práctica su uso en Pytorch.

##  AD en TensorFlow

Si no tenemos TensorFlow en nuestro entorno virtual, entonces hemos de incorporarlo al kernel de Python. Esto lo podemos hacer desde el Powershell Prompt de Anaconda escribiendo:

**conda create -n tf tensorflow**

**conda activate tf**

A continuación cambiamos al kernel de tf y ya podemos importar TensorFlow



In [9]:
import tensorflow as tf

[tf.autodiff](https://www.tensorflow.org/api_docs/python/tf/autodiff) es el módulo de TensorFlow para AD. Se compone de dos clases:

1) [tf.autodiff.ForwardAccumulator](https://www.tensorflow.org/api_docs/python/tf/autodiff/ForwardAccumulator) para el modo forward, y

2) [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) para el modo backward.

Nos centramos a continuación en la clase **backward** con [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) .

Veamos un primer ejemplo elemental: derivada de la función $y = x^2$ en $x=3$.

In [10]:
 
x = tf.Variable(3.0) # asigna el valor 3 a la variable x

with tf.GradientTape() as tape: # almacena todas las operaciones hechas con tf.GradientTape en tape
  y = x ** 2

dy_dx = tape.gradient(y, x) # derivada de y respecto a x
print(f"derivada de y respecto a x en x=3 igual a {dy_dx}")


derivada de y respecto a x en x=3 igual a 6.0


Para calcular derivadas de orden superior procedemos así:

In [11]:
with tf.GradientTape() as tape2: # almacena todas las operaciones hechas con tf.GradientTape en tape
  with tf.GradientTape() as tape1:
    y = x ** 3
  dy_dx = tape1.gradient(y,x)  # derivada de y respecto a x
d2y_d2x = tape2.gradient(dy_dx, x) # derivada segunda de y respecto a x dos veces

print(f"derivada de y respecto a x en x=3 igual a {dy_dx}")
print(f"derivada segunda de y respecto a x dos veces en x=3 igual a {d2y_d2x}")


derivada de y respecto a x en x=3 igual a 27.0
derivada segunda de y respecto a x dos veces en x=3 igual a 18.0


Consideremos ahora una función de dos variables

$$
f(x_1, x_2) = x_1^2(x_1 + x_2)
$$

Vamos a calcular su gradiente en $x_1 = 2$, $x_2 = 3$. 

In [12]:
x = tf.Variable([2.0, 3.0]) 

print(x)

with tf.GradientTape() as tape: # almacena todas las operaciones hechas con tf.GradientTape en tape
  f = x[0]** 2 * (x[0] + x[1])

print(f"f(2, 3) = {f}")

grad_f = tape.gradient(f, x) # gradiente de f
print(f"gradiente de f en (2, 3) = {grad_f}")


<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
f(2, 3) = 20.0
gradiente de f en (2, 3) = [24.  4.]


La matriz Hessiana y sus valores propios se pueden calcular así:

In [13]:
with tf.GradientTape() as tape2: # almacenamos todas las operaciones hechas con tf.GradientTape en tape2
    with tf.GradientTape() as tape1:
        f = x[0] ** 2 * (x[0] + x[1])
    grad_f = tape1.gradient(f, x) # gradiente de f
hess_f = tape2.jacobian(grad_f, x) # hessiana de f
print(f"matriz hessiana de f en (2, 3) = \n {hess_f}")

from numpy.linalg import eig

eigenvalues, _ = eig(hess_f)

print(f"valores propios de la matriz hessiana de f en (2, 3) = \n {eigenvalues}")



matriz hessiana de f en (2, 3) = 
 [[18.  4.]
 [ 4.  0.]]
valores propios de la matriz hessiana de f en (2, 3) = 
 [18.848858  -0.8488578]


Veamos ahora un modelo más elaborado. En concreto, vamos a calcular el gradiente de 
$$
y = x * w  + b
$$
donde $x$ es un vector fila de $4$ componentes, $w$ es una matriz $4\times 3$ y $b$ un vector columna de $3$ componentes. Obsérvese que $y$ es una función de $\mathbb{R}^4 \to \mathbb{R}^3$, ya que 
$$
y = x * w  + b = (x_1, x_2, x_3, x_4) 
\left(
    \begin{array}{ccc}
    w_{11} & w_{12} & w_{13} \\
    w_{21} & w_{22} & w_{23} \\
    w_{31} & w_{32} & w_{33} \\
    w_{41} & w_{42} & w_{43} 
    \end{array}
\right) + 
\begin{pmatrix}
b_1 \\ b_2 \\ b_3
\end{pmatrix}
= 
\begin{pmatrix}
x_1 w_{11} + x_2 w_{21} + x_3 w_{31} + x_4 w_{41} +b_1\\
x_1 w_{12} + x_2 w_{22} + x_3 w_{32} + x_4 w_{42} +b_2\\
x_1 w_{13} + x_2 w_{23} + x_3 w_{33} + x_4 w_{43} +b_3\\
\end{pmatrix}
$$

In [14]:
# empezamos borrando tape
del tape

In [54]:
# A continuación, definimos las variables asignando valores aleatorios a los pesos
# y unos a los biases
tensor_x = tf.Variable([[1, 2, 3, 4]], dtype=tf.float32, name='x')
tensor_w = tf.Variable(tf.random.uniform((4, 3), minval=-20, maxval=20, 
                       dtype=tf.float32), name='w') # w: weight
tensor_b = tf.Variable(tf.ones(3, dtype=tf.float32), name='b') # bias
print(tensor_x)
print(tensor_w)
print(tensor_b)

<tf.Variable 'x:0' shape=(1, 4) dtype=float32, numpy=array([[1., 2., 3., 4.]], dtype=float32)>
<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
array([[ 14.927208 ,  16.68613  ,  18.564335 ],
       [  3.6685467,   4.7894325,  -6.357937 ],
       [-15.565605 ,  -4.2436075, -13.076515 ],
       [  2.9025555, -13.434191 ,  -4.547262 ]], dtype=float32)>
<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>


Además de la función $y = x * w + b$ consideramos la función de pérdida

$$
\text{loss } = \frac{1}{3} \sum_{j=1}^3 (y_j - (y_{\text{label}})_j)^2
$$

In [16]:
# definimos las funciones de las que queremos calcular sus gradientes
# usaremos el atributo persistent=True para poder llamar varias veces a 
#  GradientTape.gradient(). De lo contrario, sólo lo podemos llamar una vez
y_label = tf.constant([[1, 2, 3]], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:   
  tensor_y = tensor_x @ tensor_w + tensor_b # 
  tensor_loss = tf.reduce_mean((tensor_y - y_label) **2 ) # dloss = mean(2*y) dy; dloss = (2*y[0] + 2*y[1] + 2*y[2])/3

print('tensor_y: ', tensor_y)# Usar torch.no_grad para evitar que se calcule el gradiente en los parámetros de la pérdida

print('tensor_loss: ', tensor_loss)

tensor_y:  tf.Tensor([[-145.99689  -72.59936  -95.67115]], shape=(1, 3), dtype=float32)
tensor_loss:  tf.Tensor(12303.048, shape=(), dtype=float32)


In [17]:
# calculamos ahora varios gradientes

tensor_dy_dx = tape.gradient(tensor_y, tensor_x) # gradiente de y respecto a x (derivada de y_i respecto x_j y sumar en i). Es decir, es un vector cuyas
                                                #componenes son las sumas de las columnas de la matriz 
                                                # jacobia previamente calculada, esto es, su primera componente es la suma de todas
                                                # las derivadas parciales respecto de x[0], y así sucesivamente
tensor_jac_dy_dx = tape.jacobian(tensor_y, tensor_x) # matriz jacobiana de y respecto a x
tensor_dloss_dy = tape.gradient(tensor_loss, tensor_y) # gradiente de loss respecto a y
tensor_dloss_dx = tape.gradient(tensor_loss, tensor_x) # gradiente de loss respecto a x

print('tensor_dy_dx: ', tensor_dy_dx)
print('jac_dy_dx: ', tensor_jac_dy_dx)
print('tensor_dloss_dy: ', tensor_dloss_dy)
print('tensor_dloss_dx: ', tensor_dloss_dx)

tensor_dy_dx:  tf.Tensor([[ -6.441292  -32.46366    -6.8784275 -56.315872 ]], shape=(1, 4), dtype=float32)
jac_dy_dx:  tf.Tensor(
[[[[-14.195752   -4.2817736 -18.25697   -17.366667 ]]

  [[ 19.2263    -16.567844    6.0485363 -19.458895 ]]

  [[-11.471839  -11.614046    5.3300056 -19.490309 ]]]], shape=(1, 3, 1, 4), dtype=float32)
tensor_dloss_dy:  tf.Tensor([[-97.997925 -49.732906 -65.78077 ]], shape=(1, 3), dtype=float32)
tensor_dloss_dx:  tf.Tensor([[1189.601  2007.553  1137.7219 3951.7324]], shape=(1, 4), dtype=float32)


Lo dejamos aquí. Esto es sólo una pequeña introducción.

Para saber más: [AT con TensorFlow](https://insights.willogy.io/tensorflow-part-3-automatic-differentiation/)